In [12]:
import os
import time
from samgeo import SamGeo
import json
import traceback

import ee
import geemap
import math
import sys, os
import json
from GEE_Func.S1_distor_dedicated import load_S1collection, S1_CalDistor, DEM_caculator
from GEE_Func.GEE_DataIOTrans import DataTrans, DataIO, Vector_process
from GEE_Func.GEE_CorreterAndFilters import ImageFilter, S1Corrector
from GEE_Func.GEE_Tools import S1_Cheker
from functools import partial
import traceback
from osgeo import gdal
Eq_pixels = DataTrans.Eq_pixels
from PackageDeepLearn.utils import DataIOTrans
import numpy as np

geemap.set_proxy(port=10809)
ee.Initialize()


years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
SETP_Season = ['-02-25', '-05-31', '-09-15', '-11-28', '-02-25']

SaveDir = r'D:\Dataset_and_Demo\SETP_GL_TimeSeries'
SAM_PreDir = r'D:\Dataset_and_Demo\SETP_GL_TimeSeries\Processed'


## 删除不符合要求的_Trans.tif

In [ ]:
for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(output_folder, exist_ok=True)
        os.chdir(output_folder)
        Cal_list = list(range(4898))
        
        # 存在任意则保留
        Cal_list = [i for i in Cal_list if (os.path.exists("{}_ADMeanFused_Trans.tif".format(f'{i:05d}')) 
                        or os.path.exists("{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}')))]
        
        
        for i in Cal_list:
            
            save_path = os.path.join(output_folder, "{}_ADMeanFused_Trans.tif".format(f'{i:05d}'))
            save_path2 = os.path.join(output_folder, "{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}'))

            if os.path.exists(save_path):
                array_ =  DataIOTrans.DataIO.read_IMG(save_path)
            elif os.path.exists(save_path2):
                
                save_path = save_path2
                array_ =  DataIOTrans.DataIO.read_IMG(save_path)
            else:
                print(f'Error: {save_path} or {save_path2} not found.')
                break 
            
            w,s,h = array_.shape
            if np.sum(array_[:,:,0] < 20) / (w * s) > 0.8 :
                print('remove path = {},原因是纯黑'.format(save_path))
                os.remove(save_path)
            if np.sum(array_[:,:,0] == 255)/ (w * s) > 0.5:
                print('remove path = {},原因是纯白'.format(save_path))
                os.remove(save_path)

## 删除没有_Trans.tif的SAM文件

In [ ]:
for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(output_folder, exist_ok=True)
        os.chdir(output_folder)
        Cal_list = list(range(4898))
        
        # 不存在任意则保留
        Cal_list = [i for i in Cal_list if not (os.path.exists("{}_ADMeanFused_Trans.tif".format(f'{i:05d}')) 
                        or os.path.exists("{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}')))]
        
        for i in Cal_list:
            output_path = os.path.join(sam_predict_folder, "{}_ADMeanFused_SAM.tif".format(f'{i:05d}'))
            if os.path.exists(output_path):
                print('remove path = {}'.format(output_path))
                os.remove(output_path)

## 删除所有_trans

In [ ]:
for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(output_folder, exist_ok=True)
        os.chdir(output_folder)
        Cal_list = list(range(4898))
        
        # 存在任意则保留
        Cal_list = [i for i in Cal_list if (os.path.exists("{}_ADMeanFused_Trans.tif".format(f'{i:05d}')) 
                        or os.path.exists("{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}')))]
        
        
        for i in Cal_list:
            
            save_path = os.path.join(output_folder, "{}_ADMeanFused_Trans.tif".format(f'{i:05d}'))
            save_path2 = os.path.join(output_folder, "{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}'))

            if os.path.exists(save_path):
                pass
            elif os.path.exists(save_path2):
                save_path = save_path2

            os.remove(save_path)

## 新建_Trans.tif

In [ ]:
for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(output_folder, exist_ok=True)
        os.chdir(output_folder)
        Cal_list = list(range(4898))
        Cal_list = [i for i in Cal_list if not (os.path.exists("{}_ADMeanFused_Trans.tif".format(f'{i:05d}')) 
                        or os.path.exists("{}_ADMeanFused_WithTiles_Trans.tif".format(f'{i:05d}')))]
            
        for i in Cal_list:
            
            save_path = os.path.join(output_folder, "{}_ADMeanFused.tif".format(f'{i:05d}'))
            json_path = os.path.join(output_folder, "{}_ADMeanFused_AOI.json".format(f'{i:05d}'))
            save_path2 = os.path.join(output_folder, "{}_ADMeanFused_WithTiles.tif".format(f'{i:05d}'))
            json_path2 = os.path.join(output_folder, "{}_ADMeanFused_WithTiles_AOI.json".format(f'{i:05d}'))
            
            output_path = os.path.join(sam_predict_folder, "{}_ADMeanFused_SAM.tif".format(f'{i:05d}'))
            
            if os.path.exists(save_path):
                imagePath = DataIOTrans.DataIO.TransImage_Values(save_path, transFunc=DataIOTrans.DataTrans.MinMaxBoundaryScaler,
                                            bandSave=[0, 0, 0], scale=255)
            elif os.path.exists(save_path2):
                imagePath = DataIOTrans.DataIO.TransImage_Values(save_path2, transFunc=DataIOTrans.DataTrans.MinMaxBoundaryScaler,
                                            bandSave=[0, 0, 0], scale=255)
                json_path = json_path2
            else:
                print(f'Error: {save_path} or {save_path2} not found.')
                break 
            
            print("{}已构建".format(save_path))

## 复制文件
将关系为‘空间一致’的属性，根据Sort进行复制

In [18]:
import geopandas as gpd
import glob
import shutil

gdf = gpd.read_file(r"D:\BaiduSyncdisk\02_论文相关\在写\SAM冰湖\数据\2023_05_31_to_2023_09_15_样本修正.shp")
Sorts = [int(i) for i in gdf[gdf['关系']=='空间一致']['Sort'].values]

SaveDir2 = r'D:\Dataset_and_Demo\SETP_GL_TS'
SAM_PreDir2 = r'D:\Dataset_and_Demo\SETP_GL_TS\Processed'

for year in years:
    for season_index in range(len(SETP_Season) - 1):
        START_DATE = ee.Date(year + SETP_Season[season_index])
        # Check if it's a cross-year season
        if SETP_Season[season_index] == '-11-28' and SETP_Season[season_index + 1] == '-02-25':
            next_year = str(int(year) + 1)
            END_DATE = ee.Date(next_year + SETP_Season[season_index + 1])
        else:
            END_DATE = ee.Date(year + SETP_Season[season_index + 1])
        print(f"START_DATE: {START_DATE.format('YYYY-MM-dd').getInfo()}, END_DATE: {END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        output_folder = os.path.join(SaveDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_predict_folder = os.path.join(SAM_PreDir, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        destination_folder = os.path.join(SaveDir2, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        sam_destination_folder = os.path.join(SAM_PreDir2, f"{START_DATE.format('YYYY-MM-dd').getInfo()}_to_{END_DATE.format('YYYY-MM-dd').getInfo()}")
        
        os.makedirs(destination_folder, exist_ok=True)
        os.makedirs(sam_destination_folder, exist_ok=True)
        
        os.chdir(output_folder)
        for i in Sorts:
            # 使用通配符查找匹配的文件
            pattern = "{}_*.tif".format(f'{i:05d}')
            matching_files = glob.glob(pattern)
        
            for file in matching_files:
                # 复制文件到目标文件夹
                shutil.copy(file, destination_folder)
                
        os.chdir(sam_predict_folder)
        for i in Sorts:
            # 使用通配符查找匹配的文件
            pattern = "{}_*.tif".format(f'{i:05d}')
            matching_files = glob.glob(pattern)
        
            for file in matching_files:
                # 复制文件到目标文件夹
                shutil.copy(file, sam_destination_folder)

        
        for i in Sorts:
            # 使用通配符查找匹配的文件
            pattern = "{}_*.tif".format(f'{i:05d}')
            matching_files = glob.glob(pattern)

START_DATE: 2015-02-25, END_DATE: 2015-05-31
START_DATE: 2015-05-31, END_DATE: 2015-09-15
START_DATE: 2015-09-15, END_DATE: 2015-11-28
START_DATE: 2015-11-28, END_DATE: 2016-02-25
START_DATE: 2016-02-25, END_DATE: 2016-05-31
START_DATE: 2016-05-31, END_DATE: 2016-09-15
START_DATE: 2016-09-15, END_DATE: 2016-11-28
START_DATE: 2016-11-28, END_DATE: 2017-02-25
START_DATE: 2017-02-25, END_DATE: 2017-05-31
START_DATE: 2017-05-31, END_DATE: 2017-09-15
START_DATE: 2017-09-15, END_DATE: 2017-11-28
START_DATE: 2017-11-28, END_DATE: 2018-02-25
START_DATE: 2018-02-25, END_DATE: 2018-05-31
START_DATE: 2018-05-31, END_DATE: 2018-09-15
START_DATE: 2018-09-15, END_DATE: 2018-11-28
START_DATE: 2018-11-28, END_DATE: 2019-02-25
START_DATE: 2019-02-25, END_DATE: 2019-05-31
START_DATE: 2019-05-31, END_DATE: 2019-09-15
START_DATE: 2019-09-15, END_DATE: 2019-11-28
START_DATE: 2019-11-28, END_DATE: 2020-02-25
START_DATE: 2020-02-25, END_DATE: 2020-05-31
START_DATE: 2020-05-31, END_DATE: 2020-09-15
START_DATE

FileNotFoundError: [WinError 2] 系统找不到指定的文件。: 'D:\\Dataset_and_Demo\\SETP_GL_TimeSeries\\Processed\\2024-09-15_to_2024-11-28'